In [10]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [31]:
dir = '/home/jsmith48/scratch/carbon_nano_tube/new_test/'

fname = '19_0'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.2
C = 0.0001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/ccsd_extrapolation/final_train_models_3/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [32]:
tvec = np.loadtxt('/home/jsmith48/scratch/carbon_nano_tube/new_test/'+fname+'.tvec',dtype=str)
tvec = np.array(tvec[:,1:],dtype=np.float64)

In [33]:
tvec

array([[24.885329,  0.      ,  0.      ],
       [ 0.      , 24.885329,  0.      ],
       [ 0.      ,  0.      , 12.789   ]])

In [34]:
# Load molecule
mol = read(molfile)
# L = 40.0
# mol.set_cell(([[100.0, 0, 0],
#                [0, 35.0, 0],
#                [0, 0, 35.0]]))

mol.set_cell((tvec))


mol.set_pbc((True, True, True))
#mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 5)
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.040746450424194336
-236419.61761202803
228


In [35]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=True).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 16:37:00  -236419.617612        0.1233
LBFGS:    1 16:37:00  -236419.644898        0.0948
LBFGS:    2 16:37:00  -236419.680602        0.1024
LBFGS:    3 16:37:00  -236420.178946        0.1340
LBFGS:    4 16:37:00  -236420.284326        0.0574
LBFGS:    5 16:37:00  -236420.289025        0.0177
LBFGS:    6 16:37:00  -236420.289365        0.0025
LBFGS:    7 16:37:00  -236420.289377        0.0012
LBFGS:    8 16:37:00  -236420.289475        0.0002
LBFGS:    9 16:37:00  -236420.289470        0.0001
LBFGS:   10 16:37:00  -236420.289472        0.0001
LBFGS:   11 16:37:00  -236420.289481        0.0002
LBFGS:   12 16:37:00  -236420.289483        0.0001
LBFGS:   13 16:37:00  -236420.289488        0.0002
LBFGS:   14 16:37:00  -236420.289466        0.0001
LBFGS:   15 16:37:00  -236420.289488        0.0004
LBFGS:   16 16:37:00  -236420.289473        0.0002
LBFGS:   17 16:37:00  -236420.289488        0.0002
LBFGS:   18 16:37:00  -236420.28

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
    
# Set the printer
dyn.attach(storeenergy, interval=50)

In [ ]:
start_time = time.time()
dyn.set_temperature(50. * units.kB)
dyn.run(25000)
print('[Heat1 Total time:', time.time() - start_time, 'seconds]')

start_time = time.time()
dyn.set_temperature(100. * units.kB)
dyn.run(25000)
print('[Heat2 Total time:', time.time() - start_time, 'seconds]')

dyn.set_temperature(150. * units.kB)
dyn.run(25000)

dyn.set_temperature(200. * units.kB)
dyn.run(25000)

dyn.set_temperature(250. * units.kB)
dyn.run(25000)

dyn.set_temperature(300. * units.kB)
dyn.run(25000)

dyn.set_temperature(325. * units.kB)
dyn.run(25000)

start_time = time.time()
dyn.set_temperature(325. * units.kB)
dyn.run(250000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Step: 100 Energy per atom: Epot = -692.992  Ekin = 0.000 (T=0.564K)  Etot = -692.992179 Esig = 2.931478
Step: 200 Energy per atom: Epot = -692.992  Ekin = 0.000 (T=1.170K)  Etot = -692.992180 Esig = 2.945771
Step: 300 Energy per atom: Epot = -692.992  Ekin = 0.000 (T=1.949K)  Etot = -692.992189 Esig = 2.959531
Step: 400 Energy per atom: Epot = -692.993  Ekin = 0.000 (T=2.671K)  Etot = -692.992220 Esig = 2.970791
Step: 500 Energy per atom: Epot = -692.993  Ekin = 0.000 (T=3.425K)  Etot = -692.992267 Esig = 2.971739
Step: 600 Energy per atom: Epot = -692.993  Ekin = 0.001 (T=4.316K)  Etot = -692.992317 Esig = 2.967483
Step: 700 Energy per atom: Epot = -692.993  Ekin = 0.001 (T=5.270K)  Etot = -692.992394 Esig = 2.976338
Step: 800 Energy per atom: Epot = -692.993  Ekin = 0.001 (T=6.124K)  Etot = -692.992495 Esig = 2.984113
Step: 900 Energy per atom: Epot = -692.993  Ekin = 0.001 (T=6.817K)  Etot = -692.992604 Esig = 2.996204
Step: 1000 Energy per atom: Epot = -692.994  Ekin = 0.001 (T=7.1

Step: 7800 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.812K)  Etot = -692.997015 Esig = 3.002887
Step: 7900 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.823K)  Etot = -692.997021 Esig = 3.000639
Step: 8000 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.630K)  Etot = -692.997031 Esig = 2.975905
Step: 8100 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.768K)  Etot = -692.997034 Esig = 2.973972
Step: 8200 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.666K)  Etot = -692.997048 Esig = 2.981875
Step: 8300 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.799K)  Etot = -692.997054 Esig = 2.969523
Step: 8400 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.798K)  Etot = -692.997079 Esig = 2.976521
Step: 8500 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.859K)  Etot = -692.997105 Esig = 2.982951
Step: 8600 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=4.881K)  Etot = -692.997123 Esig = 2.960476
Step: 8700 Energy per atom: Epot = -692.998  Ekin = 0.0

Step: 15500 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.265K)  Etot = -692.996934 Esig = 3.000938
Step: 15600 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.407K)  Etot = -692.996919 Esig = 3.001336
Step: 15700 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.689K)  Etot = -692.996910 Esig = 2.966216
Step: 15800 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.580K)  Etot = -692.996902 Esig = 2.949500
Step: 15900 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.750K)  Etot = -692.996898 Esig = 2.972686
Step: 16000 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.871K)  Etot = -692.996889 Esig = 2.966752
Step: 16100 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=7.109K)  Etot = -692.996875 Esig = 2.967108
Step: 16200 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=6.853K)  Etot = -692.996875 Esig = 2.990801
Step: 16300 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=7.101K)  Etot = -692.996862 Esig = 2.989446
Step: 16400 Energy per atom: Epot = -692.998  

Step: 23200 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.567K)  Etot = -692.996644 Esig = 2.961843
Step: 23300 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.412K)  Etot = -692.996663 Esig = 2.983310
Step: 23400 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.243K)  Etot = -692.996655 Esig = 2.998086
Step: 23500 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.370K)  Etot = -692.996643 Esig = 2.960080
Step: 23600 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.345K)  Etot = -692.996667 Esig = 2.949767
Step: 23700 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.470K)  Etot = -692.996686 Esig = 2.960355
Step: 23800 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.323K)  Etot = -692.996655 Esig = 2.945943
Step: 23900 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.364K)  Etot = -692.996654 Esig = 2.972537
Step: 24000 Energy per atom: Epot = -692.998  Ekin = 0.001 (T=9.558K)  Etot = -692.996649 Esig = 2.948530
Step: 24100 Energy per atom: Epot = -692.998  

Step: 30800 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.962K)  Etot = -692.996070 Esig = 2.923901
Step: 30900 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.882K)  Etot = -692.996066 Esig = 2.914445
Step: 31000 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.753K)  Etot = -692.996062 Esig = 2.949156
Step: 31100 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.765K)  Etot = -692.996075 Esig = 2.939236
Step: 31200 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.991K)  Etot = -692.996089 Esig = 2.951208
Step: 31300 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=13.177K)  Etot = -692.996044 Esig = 2.902144
Step: 31400 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=12.754K)  Etot = -692.996054 Esig = 2.944309
Step: 31500 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=13.284K)  Etot = -692.996061 Esig = 2.926409
Step: 31600 Energy per atom: Epot = -692.998  Ekin = 0.002 (T=13.300K)  Etot = -692.996085 Esig = 2.918739
Step: 31700 Energy per atom: Epot = -

...

Step: 219200 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=89.785K)  Etot = -692.984662 Esig = 2.663223
Step: 219300 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=87.633K)  Etot = -692.984680 Esig = 2.622794
Step: 219400 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=87.537K)  Etot = -692.984956 Esig = 2.636548
Step: 219500 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=88.045K)  Etot = -692.985036 Esig = 2.729858
Step: 219600 Energy per atom: Epot = -692.997  Ekin = 0.012 (T=90.050K)  Etot = -692.984936 Esig = 2.764303
Step: 219700 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=85.368K)  Etot = -692.985207 Esig = 2.733069
Step: 219800 Energy per atom: Epot = -692.997  Ekin = 0.011 (T=87.634K)  Etot = -692.985419 Esig = 2.781894
Step: 219900 Energy per atom: Epot = -692.997  Ekin = 0.011 (T=86.864K)  Etot = -692.985423 Esig = 2.655855
Step: 220000 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=85.988K)  Etot = -692.985357 Esig = 2.648253
[ANI Total time: 113.0594489

Step: 226700 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=93.988K)  Etot = -692.983849 Esig = 2.746809
Step: 226800 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=95.704K)  Etot = -692.983909 Esig = 2.667970
Step: 226900 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=90.127K)  Etot = -692.984062 Esig = 2.755694
Step: 227000 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=90.598K)  Etot = -692.984012 Esig = 2.731966
Step: 227100 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=93.577K)  Etot = -692.984019 Esig = 2.784162
Step: 227200 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=86.807K)  Etot = -692.984308 Esig = 2.737218
Step: 227300 Energy per atom: Epot = -692.995  Ekin = 0.011 (T=86.055K)  Etot = -692.984368 Esig = 2.796236
Step: 227400 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=88.022K)  Etot = -692.984390 Esig = 2.783604
Step: 227500 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=91.789K)  Etot = -692.984560 Esig = 2.723276
[ANI Total time: 112.5410590

Step: 234200 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=94.972K)  Etot = -692.983984 Esig = 2.697580
Step: 234300 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=91.311K)  Etot = -692.984026 Esig = 2.766667
Step: 234400 Energy per atom: Epot = -692.997  Ekin = 0.013 (T=96.733K)  Etot = -692.984189 Esig = 2.674186
Step: 234500 Energy per atom: Epot = -692.997  Ekin = 0.012 (T=92.335K)  Etot = -692.984585 Esig = 2.762674
Step: 234600 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=90.898K)  Etot = -692.984637 Esig = 2.812375
Step: 234700 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=90.423K)  Etot = -692.984442 Esig = 2.721498
Step: 234800 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=89.715K)  Etot = -692.984560 Esig = 2.789133
Step: 234900 Energy per atom: Epot = -692.997  Ekin = 0.012 (T=92.553K)  Etot = -692.984568 Esig = 2.729536
Step: 235000 Energy per atom: Epot = -692.996  Ekin = 0.011 (T=88.426K)  Etot = -692.984756 Esig = 2.758892
[ANI Total time: 112.8234877

Step: 241700 Energy per atom: Epot = -692.995  Ekin = 0.012 (T=95.754K)  Etot = -692.982920 Esig = 2.721475
Step: 241800 Energy per atom: Epot = -692.995  Ekin = 0.012 (T=96.398K)  Etot = -692.982858 Esig = 2.684274
Step: 241900 Energy per atom: Epot = -692.996  Ekin = 0.013 (T=99.127K)  Etot = -692.982957 Esig = 2.756935
Step: 242000 Energy per atom: Epot = -692.996  Ekin = 0.013 (T=98.713K)  Etot = -692.983138 Esig = 2.598890
Step: 242100 Energy per atom: Epot = -692.996  Ekin = 0.013 (T=98.200K)  Etot = -692.983541 Esig = 2.626917
Step: 242200 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=94.673K)  Etot = -692.983784 Esig = 2.536619
Step: 242300 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=96.284K)  Etot = -692.983525 Esig = 2.515295
Step: 242400 Energy per atom: Epot = -692.996  Ekin = 0.012 (T=96.099K)  Etot = -692.983631 Esig = 2.542777
Step: 242500 Energy per atom: Epot = -692.997  Ekin = 0.013 (T=98.786K)  Etot = -692.983774 Esig = 2.678813
[ANI Total time: 112.4582560

Step: 249200 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=99.114K)  Etot = -692.982223 Esig = 2.695096
Step: 249300 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=98.086K)  Etot = -692.982215 Esig = 2.709462
Step: 249400 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=102.081K)  Etot = -692.982067 Esig = 2.664601
Step: 249500 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=98.585K)  Etot = -692.982327 Esig = 2.673441
Step: 249600 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=100.024K)  Etot = -692.982426 Esig = 2.596970
Step: 249700 Energy per atom: Epot = -692.996  Ekin = 0.013 (T=102.238K)  Etot = -692.982297 Esig = 2.671107
Step: 249800 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=101.805K)  Etot = -692.982301 Esig = 2.637224
Step: 249900 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=97.265K)  Etot = -692.982210 Esig = 2.596770
Step: 250000 Energy per atom: Epot = -692.995  Ekin = 0.013 (T=100.747K)  Etot = -692.981931 Esig = 2.595058
[ANI Total time: 112.94